In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
58,application_1607949680860_0060,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import json
import numpy as np
from hops import model
from hops.model import Metric

# Query Model Repository for best anomaly detection model

In [3]:
MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

In [4]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MIN)

In [5]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: ganAml
Model version: 1
{'loss': '-0.5314414501190186'}

# Create Model Serving of Exported Model

In [6]:
from hops import serving

In [7]:
MODEL_NAME

'ganAml'

In [8]:
# Create serving
model_path="/Models/" + best_model['name']
model_path
response = serving.create_or_update(artifact_path=model_path, serving_name=MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

Creating a serving for model ganAml ...
Serving for model ganAml successfully created

In [9]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

ganAml

In [10]:
# Get serving status
serving.get_status(MODEL_NAME)

'Stopped'

# Start Model Serving Server

In [11]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

Starting serving with name: ganAml...
Serving with name: ganAml successfully started

In [12]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [13]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [14]:
eval_td = fs.get_training_dataset("gan_eval_df", 1)

In [15]:
eval_td.show(5)

+------+--------------------+
|target|           embedding|
+------+--------------------+
|     1|[-0.5829833, -0.9...|
|     1|[-1.0027698, -0.1...|
|     1|[-0.49065784, 0.8...|
|     1|[-0.20831347, 0.0...|
|     1|[-0.78685, -0.722...|
+------+--------------------+
only showing top 5 rows

In [16]:
def model_server(model_name, input):
    data = {"signature_name": "serving_default", "inputs": [input]}
    return serving.make_inference_request(model_name, data)['outputs']

In [17]:
scored_df = eval_td.read().rdd.map(lambda x: (x.target,model_server(MODEL_NAME, np.array(x.embedding).tolist()))).toDF()

In [18]:
scored_df.show()

+---+------------+
| _1|          _2|
+---+------------+
|  1|[5.71745777]|
|  1|[7.18372154]|
|  1|[5.38492537]|
|  1|[6.28217459]|
|  1| [5.3201375]|
|  1|[4.60506916]|
|  1|[7.14408731]|
|  1|[6.32788944]|
|  1|[8.08600521]|
|  1|[8.99386883]|
|  1|[3.49662709]|
|  1|[3.76946759]|
|  1|[4.40817595]|
|  1|[4.71811628]|
|  1|   [5.79049]|
|  1|[2.50109863]|
|  1|[4.33767557]|
|  1|[14.1510382]|
|  1|[8.21180153]|
|  1|[4.12480402]|
+---+------------+
only showing top 20 rows